In [1]:
import importlib, sys, time, numpy as np
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm

# Ensure the sorth module is imported correctly
if 'sorth' in sys.modules:
    importlib.reload(sys.modules['sorth'])
else:
    import sorth                          
from sorth import Sort, KalmanBoxTracker, associate_detections_to_trackers

from ultralytics import YOLO

c:\Users\hridi\anaconda3\envs\CV2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_1 = YOLO("model/best.pt")

In [ ]:
def evaluate_model_on_video(video_path, model, dist_th=120, max_age=12, conf_th=0.20, output_base_dir="Output"):
    """
    Evaluate a model on a single video and save results.
    
    """
    
    video_path = Path(video_path)
    
    # Create output directories
    frames_dir = Path(f"{output_base_dir}/frames/{video_path.stem}")
    frames_dir.mkdir(parents=True, exist_ok=True)
    
    csv_dir = Path(f"{output_base_dir}/eval/")
    csv_dir.mkdir(parents=True, exist_ok=True)
    
    # Initialize tracker
    KalmanBoxTracker.count = 0
    tracker = Sort(max_age=max_age, min_hits=1, dist_threshold=dist_th)
    
    # Open video
    cap = cv2.VideoCapture(str(video_path))
    
    # Colors and font
    CLR_DET, CLR_PRED, CLR_TRACK, CLR_LINE = (0,255,0), (255,0,0), (255,0,255), (0,255,255)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    frame_idx, t0 = 0, time.time()
    records = []
    
    print(f"Processing {video_path.name}...")
    
    # Main processing loop
    while True:
        ok, frame = cap.read()
        if not ok:
            break
            
        # YOLO detections
        result = model(frame, verbose=False)[0]
        xyxy = result.boxes.xyxy.cpu().numpy() if result.boxes else np.empty((0,4))
        conf = result.boxes.conf.cpu().numpy() if result.boxes else np.empty((0,))
        keep = conf > conf_th
        dets = xyxy[keep]
        conf = conf[keep]
        dets_in = np.hstack([dets, conf[:,None]]) if dets.size else np.empty((0,5))
        
        # Save Kalman predictions before update
        preds_before = []
        for trk in tracker.trackers:
            pred = trk.predict()[0]
            preds_before.append(pred)
            
        # Run tracker update
        tracks = tracker.update(dets_in)
        for (x1, y1, x2, y2, tid) in tracks:
            cx = round((x1 + x2) / 2, 6)
            cy = round((y1 + y2) / 2, 6)
            records.append({
                "t": frame_idx,
                "hexbug": int(tid),
                "x": cx,
                "y": cy
            })
            
        # Draw visualization
        canvas = frame.copy()
        
        # Draw detections (green)
        for (x1,y1,x2,y2,sc) in dets_in:
            cv2.rectangle(canvas,(int(x1),int(y1)),(int(x2),int(y2)), CLR_DET,2)
            
        # Draw Kalman predictions (blue)
        for (x1,y1,x2,y2) in preds_before:
            cv2.rectangle(canvas,(int(x1),int(y1)),(int(x2),int(y2)), CLR_PRED,1)
            
        # Draw tracks (magenta)
        for (x1,y1,x2,y2,tid) in tracks:
            cv2.rectangle(canvas,(int(x1),int(y1)),(int(x2),int(y2)), CLR_TRACK,2)
            cx,cy = int((x1+x2)/2), int((y1+y2)/2)
            cv2.putText(canvas,f"{int(tid)}",(cx+5,cy-5),font,0.6,CLR_TRACK,2)
            
        # Draw match lines (yellow)
        if preds_before and dets_in.size:
            tmp_trk = np.zeros((len(preds_before),5))
            tmp_trk[:,:4] = np.array(preds_before)
            matches,_,_ = associate_detections_to_trackers(
                dets_in, tmp_trk, dist_threshold=dist_th)
            for d,t in matches:
                dx,dy = (dets_in[d,0]+dets_in[d,2])/2, (dets_in[d,1]+dets_in[d,3])/2
                tx,ty = (preds_before[t][0]+preds_before[t][2])/2, (preds_before[t][1]+preds_before[t][3])/2
                cv2.line(canvas,(int(dx),int(dy)),(int(tx),int(ty)),CLR_LINE,1,cv2.LINE_AA)
                
        cv2.putText(canvas,f"frame {frame_idx}",(20,30),font,1,(255,255,255),2)
        
        # Save frame
        cv2.imwrite(str(frames_dir / f"{frame_idx:05d}.jpg"), canvas)
        frame_idx += 1
        
    cap.release()
    
    # Save CSV in correct format to match ground truth structure
    csv_path = csv_dir / f"{video_path.stem}.csv"
    df = pd.DataFrame(records).sort_values(['t', 'hexbug'])
    
    # Reset index to create the first unnamed column like in ground truth
    df = df.reset_index(drop=True)
    
    # Save with index=True to create the first column, but no index_label
    df.to_csv(csv_path, index=True)
  
    
    print(f"{video_path.name}: {len(records)} records, {frame_idx} frames")
    print(f"  CSV: {csv_path}")
    print(f"  Frames: {frames_dir}")
    
    return {
        'video': video_path.name,
        'csv_path': csv_path,
        'frames_dir': frames_dir,
        'num_records': len(records),
        'num_frames': frame_idx,
    }

In [5]:
video_dir = Path("Validation") # path to test videos
output_base1 = "Val"  # output dir

# Parameters for evaluation
eval_params = {
    'dist_th': 120,
    'max_age': 12, 
    'conf_th': 0.20,
    'output_base_dir': output_base1
}


print(f"Parameters: {eval_params}")
print(f"Output directory: {output_base1}")
print("=" * 50)


# Get all video files
video_files = sorted(video_dir.glob("*.mp4"))
print(f"Found {len(video_files)} video files to process")

results = []

for i, video_path in enumerate(video_files, 1):
    print(f"\n[{i}/{len(video_files)}] Processing {video_path.name}")
    
    try:
        result = evaluate_model_on_video(
            video_path=video_path,
            model=model_1,  # Use model_1, change to model_2 if needed
            **eval_params
        )
        results.append(result)
        
    except Exception as e:
        print(f"❌ Error processing {video_path.name}: {e}")
        results.append({
            'video': video_path.name,
            'error': str(e),
            'num_records': 0,
            'num_frames': 0,
            'fps': 0
        })



Parameters: {'dist_th': 120, 'max_age': 12, 'conf_th': 0.2, 'output_base_dir': 'Val'}
Output directory: Val
Found 5 video files to process

[1/5] Processing training011.mp4
Processing training011.mp4...
training011.mp4: 202 records, 101 frames
  CSV: Val\eval\training011.csv
  Frames: Val\frames\training011

[2/5] Processing training024.mp4
Processing training024.mp4...
training024.mp4: 101 records, 101 frames
  CSV: Val\eval\training024.csv
  Frames: Val\frames\training024

[3/5] Processing training034.mp4
Processing training034.mp4...
training034.mp4: 303 records, 101 frames
  CSV: Val\eval\training034.csv
  Frames: Val\frames\training034

[4/5] Processing training071.mp4
Processing training071.mp4...
training071.mp4: 195 records, 101 frames
  CSV: Val\eval\training071.csv
  Frames: Val\frames\training071

[5/5] Processing training082.mp4
Processing training082.mp4...
training082.mp4: 196 records, 101 frames
  CSV: Val\eval\training082.csv
  Frames: Val\frames\training082


## Run with preprocess

In [6]:
import cv2
import numpy as np

def is_dark(frame_bgr: np.ndarray, thr: int = 80) -> bool:
    """
    Return True if the frame’s median V-channel < thr (0-255 scale).
    thr = 80 works well for 8-bit footage where ‘normal’ indoor lighting
    has median V around 110-140.
    """
    v = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)[:, :, 2]
    return np.median(v) < thr

def normalise_exposure(img_bgr: np.ndarray) -> np.ndarray:
    """
    1. 1st–99th-percentile stretch on V channel
    2. CLAHE local contrast boost
    3. Mild saturation equalisation
    """
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

    # ---- 1. robust rescale ---------------------------------------------
    v = hsv[:, :, 2].astype(np.float32)
    lo, hi = np.percentile(v, (1, 99))
    if hi - lo > 1:                       # avoid div-by-zero
        v = np.clip((v - lo) * 255.0 / (hi - lo), 0, 255)
    hsv[:, :, 2] = v.astype(np.uint8)

    # ---- 2. local CLAHE -------------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv[:, :, 2] = clahe.apply(hsv[:, :, 2])

    # ---- 3. balance colour cast ----------------------------------------
    hsv[:, :, 1] = cv2.equalizeHist(hsv[:, :, 1])

    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)


In [7]:
def evaluate_model_on_video(video_path, model, dist_th=120, max_age=12, conf_th=0.20, output_base_dir="Output"):
    """
    Evaluate a model on a single video and save results.
    
    Args:
        video_path: Path to the video file
        model: YOLO model to use for detection
        dist_th: Distance threshold for tracking (pixels)
        max_age: Maximum age for tracks
        conf_th: Confidence threshold for YOLO detections
        output_base_dir: Base directory for outputs
    
    Returns:
        dict: Results including CSV path, frame count, FPS
    """
    
    video_path = Path(video_path)
    
    # Create output directories
    frames_dir = Path(f"{output_base_dir}/frames/{video_path.stem}")
    frames_dir.mkdir(parents=True, exist_ok=True)
    
    csv_dir = Path(f"{output_base_dir}/eval/{video_path.stem}")
    csv_dir.mkdir(parents=True, exist_ok=True)
    
    # Initialize tracker
    KalmanBoxTracker.count = 0
    tracker = Sort(max_age=max_age, min_hits=1, dist_threshold=dist_th)
    
    # Open video
    cap = cv2.VideoCapture(str(video_path))
    
    # Colors and font
    CLR_DET, CLR_PRED, CLR_TRACK, CLR_LINE = (0,255,0), (255,0,0), (255,0,255), (0,255,255)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    frame_idx, t0 = 0, time.time()
    records = []
    
    print(f"Processing {video_path.name}...")
    
    # Main processing loop
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        
        # Brighten dark frames
        if is_dark(frame, thr=80):            # choose your threshold once
            frame = normalise_exposure(frame)
            
        # YOLO detections
        result = model(frame, verbose=False)[0]
        xyxy = result.boxes.xyxy.cpu().numpy() if result.boxes else np.empty((0,4))
        conf = result.boxes.conf.cpu().numpy() if result.boxes else np.empty((0,))
        keep = conf > conf_th
        dets = xyxy[keep]
        conf = conf[keep]
        dets_in = np.hstack([dets, conf[:,None]]) if dets.size else np.empty((0,5))
        
        # Save Kalman predictions before update
        preds_before = []
        for trk in tracker.trackers:
            pred = trk.predict()[0]
            preds_before.append(pred)
            
        # Run tracker update
        tracks = tracker.update(dets_in)
        for (x1, y1, x2, y2, tid) in tracks:
            cx = round((x1 + x2) / 2, 6)
            cy = round((y1 + y2) / 2, 6)
            records.append({
                "t": frame_idx,
                "hexbug": int(tid),
                "x": cx,
                "y": cy
            })
            
        # Draw visualization
        canvas = frame.copy()
        
        # Draw detections (green)
        for (x1,y1,x2,y2,sc) in dets_in:
            cv2.rectangle(canvas,(int(x1),int(y1)),(int(x2),int(y2)), CLR_DET,2)
            
        # Draw Kalman predictions (blue)
        for (x1,y1,x2,y2) in preds_before:
            cv2.rectangle(canvas,(int(x1),int(y1)),(int(x2),int(y2)), CLR_PRED,1)
            
        # Draw tracks (magenta)
        for (x1,y1,x2,y2,tid) in tracks:
            cv2.rectangle(canvas,(int(x1),int(y1)),(int(x2),int(y2)), CLR_TRACK,2)
            cx,cy = int((x1+x2)/2), int((y1+y2)/2)
            cv2.putText(canvas,f"{int(tid)}",(cx+5,cy-5),font,0.6,CLR_TRACK,2)
            
        # Draw match lines (yellow)
        if preds_before and dets_in.size:
            tmp_trk = np.zeros((len(preds_before),5))
            tmp_trk[:,:4] = np.array(preds_before)
            matches,_,_ = associate_detections_to_trackers(
                dets_in, tmp_trk, dist_threshold=dist_th)
            for d,t in matches:
                dx,dy = (dets_in[d,0]+dets_in[d,2])/2, (dets_in[d,1]+dets_in[d,3])/2
                tx,ty = (preds_before[t][0]+preds_before[t][2])/2, (preds_before[t][1]+preds_before[t][3])/2
                cv2.line(canvas,(int(dx),int(dy)),(int(tx),int(ty)),CLR_LINE,1,cv2.LINE_AA)
                
        cv2.putText(canvas,f"frame {frame_idx}",(20,30),font,1,(255,255,255),2)
        
        # Save frame
        cv2.imwrite(str(frames_dir / f"{frame_idx:05d}.jpg"), canvas)
        frame_idx += 1
        
    cap.release()
    
    # Save CSV in correct format to match ground truth structure
    csv_path = csv_dir / f"{video_path.stem}.csv"
    df = pd.DataFrame(records).sort_values(['t', 'hexbug'])
    
    # Reset index to create the first unnamed column like in ground truth
    df = df.reset_index(drop=True)
    
    # Save with index=True to create the first column, but no index_label
    df.to_csv(csv_path, index=True)
    
    # fps = frame_idx / (time.time() - t0)
    
    print(f"{video_path.name}: {len(records)} records, {frame_idx} frames")
    print(f"  CSV: {csv_path}")
    print(f"  Frames: {frames_dir}")
    
    return {
        'video': video_path.name,
        'csv_path': csv_path,
        'frames_dir': frames_dir,
        'num_records': len(records),
        'num_frames': frame_idx,
    }

In [8]:
video_dir = Path("Leaderboard_data") # path to test videos
output_base1 = "Output_l"  # output dir

# Parameters for evaluation
eval_params = {
    'dist_th': 120,
    'max_age': 12, 
    'conf_th': 0.20,
    'output_base_dir': output_base1
}


print(f"Parameters: {eval_params}")
print(f"Output directory: {output_base1}")
print("=*50")


# Get all video files
video_files = sorted(video_dir.glob("*.mp4"))
print(f"Found {len(video_files)} video files to process")

results = []

for i, video_path in enumerate(video_files, 1):
    print(f"\n[{i}/{len(video_files)}] Processing {video_path.name}")
    
    try:
        result = evaluate_model_on_video(
            video_path=video_path,
            model=model_1,  # Use model_1, change to model_2 if needed
            **eval_params
        )
        results.append(result)
        
    except Exception as e:
        print(f"❌ Error processing {video_path.name}: {e}")
        results.append({
            'video': video_path.name,
            'error': str(e),
            'num_records': 0,
            'num_frames': 0,
            'fps': 0
        })



Parameters: {'dist_th': 120, 'max_age': 12, 'conf_th': 0.2, 'output_base_dir': 'Output_l'}
Output directory: Output_l
=*50
Found 5 video files to process

[1/5] Processing test001.mp4
Processing test001.mp4...
test001.mp4: 303 records, 101 frames
  CSV: Output_l\eval\test001\test001.csv
  Frames: Output_l\frames\test001

[2/5] Processing test002.mp4
Processing test002.mp4...
test002.mp4: 101 records, 101 frames
  CSV: Output_l\eval\test002\test002.csv
  Frames: Output_l\frames\test002

[3/5] Processing test003.mp4
Processing test003.mp4...
test003.mp4: 390 records, 101 frames
  CSV: Output_l\eval\test003\test003.csv
  Frames: Output_l\frames\test003

[4/5] Processing test004.mp4
Processing test004.mp4...
test004.mp4: 303 records, 101 frames
  CSV: Output_l\eval\test004\test004.csv
  Frames: Output_l\frames\test004

[5/5] Processing test005.mp4
Processing test005.mp4...
test005.mp4: 101 records, 101 frames
  CSV: Output_l\eval\test005\test005.csv
  Frames: Output_l\frames\test005


In [15]:
from pathlib import Path
import time, cv2, numpy as np, pandas as pd
  # ← gives letterbox + scale_boxes

# ── helper: quick brightness test ──────────────────────────────
def is_dark(bgr, thr=80):
    v = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)[:, :, 2]
    return np.median(v) < thr

def letterbox(img, new_shape=640, color=(114, 114, 114)):
    """Resize and pad image to meet stride-multiple constraints."""
    import math, cv2, numpy as np
    h, w = img.shape[:2]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # scale ratio (new / old)  and compute padding
    r = min(new_shape[0] / h, new_shape[1] / w)
    new_unpad = (int(round(w * r)), int(round(h * r)))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]
    dw, dh = dw // 2, dh // 2

    # resize
    if (w, h) != new_unpad:
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)

    # pad
    img = cv2.copyMakeBorder(img, dh, dh, dw, dw, cv2.BORDER_CONSTANT, value=color)
    return img, r, (dw, dh)

def scale_boxes(img_shape, boxes, ori_shape, ratio, pad):
    """Invert letter-box transform on a set of xyxy boxes."""
    boxes[:, [0, 2]] -= pad[0]
    boxes[:, [1, 3]] -= pad[1]
    boxes[:, :4] /= ratio
    boxes[:, 0::2] = boxes[:, 0::2].clip(0, ori_shape[1])
    boxes[:, 1::2] = boxes[:, 1::2].clip(0, ori_shape[0])
    return boxes


def evaluate_model_on_video(
        video_path, model,
        dist_th=120, max_age=12, conf_th=0.20,
        output_base_dir="Output"):

    video_path = Path(video_path)

    # ── output dirs ────────────────────────────────────────────
    frames_dir = Path(f"{output_base_dir}/frames/{video_path.stem}")
    frames_dir.mkdir(parents=True, exist_ok=True)
    csv_dir = Path(f"{output_base_dir}/eval/{video_path.stem}")
    csv_dir.mkdir(parents=True, exist_ok=True)

    KalmanBoxTracker.count = 0
    tracker = Sort(max_age=max_age, min_hits=1, dist_threshold=dist_th)

    cap = cv2.VideoCapture(str(video_path))
    CLR_DET, CLR_PRED, CLR_TRACK, CLR_LINE = (0,255,0), (255,0,0), (255,0,255), (0,255,255)
    font = cv2.FONT_HERSHEY_SIMPLEX

    frame_idx, records = 0, []
    print(f"Processing {video_path.name} …")

    # ───────────────────────── main loop ───────────────────────
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        # — 1. optional exposure fix —
        if is_dark(frame, thr=80):
            frame = normalise_exposure(frame)

        # — 2. letter-box to square, feed YOLO —
        sq, ratio, (dw, dh) = letterbox(frame, new_shape=640)
        yolo_out = model(sq, verbose=False)[0]

        # — 3. move boxes back to original frame coords —
        boxes = yolo_out.boxes.cpu()
        xyxy = boxes.xyxy.numpy()                # (N,4)
        conf = boxes.conf.numpy()
        xyxy = scale_boxes(sq.shape, xyxy, frame.shape,
                               ratio=ratio, pad=(dw, dh))

        keep = conf > conf_th
        dets_in = np.hstack([xyxy[keep], conf[keep, None]]) if keep.any() else np.empty((0,5))

        # — 4. predict & update tracker —
        preds_before = [trk.predict()[0] for trk in tracker.trackers]
        tracks = tracker.update(dets_in)

        # — 5. store centres for CSV —
        for (x1, y1, x2, y2, tid) in tracks:
            records.append({"t": frame_idx,
                            "hexbug": int(tid),
                            "x": round((x1+x2)/2, 6),
                            "y": round((y1+y2)/2, 6)})

        # — 6. optional on-screen drawing —
        canvas = frame.copy()
        for (x1,y1,x2,y2,sc) in dets_in:
            cv2.rectangle(canvas, (int(x1),int(y1)), (int(x2),int(y2)), CLR_DET, 2)
        for (x1,y1,x2,y2) in preds_before:
            cv2.rectangle(canvas, (int(x1),int(y1)), (int(x2),int(y2)), CLR_PRED, 1)
        for (x1,y1,x2,y2,tid) in tracks:
            cv2.rectangle(canvas,(int(x1),int(y1)),(int(x2),int(y2)), CLR_TRACK,2)
            cx,cy = int((x1+x2)/2), int((y1+y2)/2)
            cv2.putText(canvas,f"{int(tid)}",(cx+5,cy-5),font,0.6,CLR_TRACK,2)

        # yellow match lines (optional eye-candy)
        if preds_before and dets_in.size:
            tmp = np.zeros((len(preds_before),5)); tmp[:,:4] = np.array(preds_before)
            matches,_,_ = associate_detections_to_trackers(dets_in, tmp, dist_threshold=dist_th)
            for d,t in matches:
                dx,dy = (dets_in[d,0]+dets_in[d,2])/2, (dets_in[d,1]+dets_in[d,3])/2
                tx,ty = (preds_before[t][0]+preds_before[t][2])/2, (preds_before[t][1]+preds_before[t][3])/2
                cv2.line(canvas,(int(dx),int(dy)),(int(tx),int(ty)),CLR_LINE,1,cv2.LINE_AA)

        cv2.putText(canvas,f"frame {frame_idx}",(20,30),font,1,(255,255,255),2)
        cv2.imwrite(str(frames_dir / f"{frame_idx:05d}.jpg"), canvas)
        frame_idx += 1

    cap.release()

    # ── write CSV ──────────────────────────────────────────────
    df = (pd.DataFrame(records)
            .sort_values(['t','hexbug'])
            .reset_index(drop=True))
    csv_path = csv_dir / f"{video_path.stem}.csv"
    df.to_csv(csv_path, index=True)      # keeps first ‘index’ column

    print(f"{video_path.name}: {len(records)} records, {frame_idx} frames")
    print(f"  CSV: {csv_path}")
    print(f"  Frames: {frames_dir}")

    return {"video": video_path.name,
            "csv_path": csv_path,
            "frames_dir": frames_dir,
            "num_records": len(records),
            "num_frames": frame_idx}


In [16]:
video_dir = Path("Leaderboard_data") # path to test videos
output_base1 = "Output_l"  # output dir

# Parameters for evaluation
eval_params = {
    'dist_th': 120,
    'max_age': 12, 
    'conf_th': 0.20,
    'output_base_dir': output_base1
}


print(f"Parameters: {eval_params}")
print(f"Output directory: {output_base1}")
print("=*50")


# Get all video files
video_files = sorted(video_dir.glob("*.mp4"))
print(f"Found {len(video_files)} video files to process")

results = []

for i, video_path in enumerate(video_files, 1):
    print(f"\n[{i}/{len(video_files)}] Processing {video_path.name}")
    
    try:
        result = evaluate_model_on_video(
            video_path=video_path,
            model=model_1,  # Use model_1, change to model_2 if needed
            **eval_params
        )
        results.append(result)
        
    except Exception as e:
        print(f"❌ Error processing {video_path.name}: {e}")
        results.append({
            'video': video_path.name,
            'error': str(e),
            'num_records': 0,
            'num_frames': 0,
            'fps': 0
        })



Parameters: {'dist_th': 120, 'max_age': 12, 'conf_th': 0.2, 'output_base_dir': 'Output_l'}
Output directory: Output_l
=*50
Found 5 video files to process

[1/5] Processing test001.mp4
Processing test001.mp4 …
test001.mp4: 303 records, 101 frames
  CSV: Output_l\eval\test001\test001.csv
  Frames: Output_l\frames\test001

[2/5] Processing test002.mp4
Processing test002.mp4 …
test002.mp4: 101 records, 101 frames
  CSV: Output_l\eval\test002\test002.csv
  Frames: Output_l\frames\test002

[3/5] Processing test003.mp4
Processing test003.mp4 …
test003.mp4: 400 records, 101 frames
  CSV: Output_l\eval\test003\test003.csv
  Frames: Output_l\frames\test003

[4/5] Processing test004.mp4
Processing test004.mp4 …
test004.mp4: 303 records, 101 frames
  CSV: Output_l\eval\test004\test004.csv
  Frames: Output_l\frames\test004

[5/5] Processing test005.mp4
Processing test005.mp4 …
test005.mp4: 101 records, 101 frames
  CSV: Output_l\eval\test005\test005.csv
  Frames: Output_l\frames\test005


In [6]:
import cv2

def select_model(video_path, threshold=3):
    """
    Selects a model based on aspect ratio of the video.
    
    Args:
        video_path (str): Path to input video
        model_M: Model for stretched (tall) videos
        model_S: Model for normal videos
        threshold (float): Aspect ratio threshold to decide stretched vs normal
    
    Returns:
        selected_model
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Could not open video")
    
    # Read first frame dimensions
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        raise ValueError("Could not read frame")
    
    h, w = frame.shape[:2]
    aspect_ratio = h / w
    
    print(f"Aspect Ratio: {aspect_ratio:.2f}")
    
    if aspect_ratio > threshold:
        print("Using Model M (stretched video).")
    
    else:
        print("Using Model S (normal video).")


In [7]:
# Get all video files
video_dir = Path("Add") # path to test videos
video_files = sorted(video_dir.glob("*.mp4"))
print(f"Found {len(video_files)} video files to process")

results = []

for i, video_path in enumerate(video_files, 1):
    print(f"\n[{i}/{len(video_files)}] Processing {video_path.name}")
    selected_model = select_model(video_path)


Found 20 video files to process

[1/20] Processing test001.mp4
Aspect Ratio: 1.41
Using Model S (normal video).

[2/20] Processing test002.mp4
Aspect Ratio: 1.31
Using Model S (normal video).

[3/20] Processing test003.mp4
Aspect Ratio: 0.69
Using Model S (normal video).

[4/20] Processing test004.mp4
Aspect Ratio: 1.12
Using Model S (normal video).

[5/20] Processing test005.mp4
Aspect Ratio: 3.57
Using Model M (stretched video).

[6/20] Processing test006.mp4
Aspect Ratio: 1.30
Using Model S (normal video).

[7/20] Processing test007.mp4
Aspect Ratio: 0.88
Using Model S (normal video).

[8/20] Processing test008.mp4
Aspect Ratio: 1.71
Using Model S (normal video).

[9/20] Processing test009.mp4
Aspect Ratio: 0.56
Using Model S (normal video).

[10/20] Processing test010.mp4
Aspect Ratio: 1.28
Using Model S (normal video).

[11/20] Processing test011.mp4
Aspect Ratio: 0.67
Using Model S (normal video).

[12/20] Processing test012.mp4
Aspect Ratio: 1.30
Using Model S (normal video).

[